### install pandas numpy matplotlib

In [ ]:

%pip install pandas numpy matplotlib

### Import pandas and read Csv

In [ ]:

# TODO: Importera pandas
import pandas as pd

!pwd
%cd "../data"
# TODO: Ladda in data från CSV-fil
df = pd.read_csv('winequality-red.csv')

# TODO: Visa första 5 raderna
df.head()

# TODO: Visa antal rader och kolumner
df.shape

# TODO: Lista alla kolumner
df.columns


### plot for show data

In [ ]:
import matplotlib.pyplot as plt

def find_outlier_data(df, columns):
    n = len(columns)
    fig, axes = plt.subplots(n, 1, figsize=(8, 3*n))  # en kolumn av subplots

    for ax, col in zip(axes, columns):
        ax.plot(df[col])
        ax.set_title(col)

    plt.tight_layout()
    plt.show()

find_outlier_data(df, df.columns)


### check nan value


In [ ]:
check_nans=df.isna().sum()
print(check_nans)


### Outliers


In [ ]:

# TODO: Visa statistik för numeriska kolumner
#print(df.describe())

# TODO: Kolla efter outliers (extremvärden)
df_test = (df.loc[:, df.columns != 'quality']).copy()
# print(f"total rows: {len(df)}")

# print(df_test.loc[:, df_test.columns != 'quality'])
for col in df_test.columns:
    Q1 = df_test[f"{col}"].quantile(0.25)
    Q3 = df_test[f"{col}"].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    #print(f"col: {col} --- IQR: {IQR} --- low: {lower_bound} --- high: {upper_bound}")

    outlier = df_test[(df_test[f"{col}"] > lower_bound) | (df_test[f"{col}"] < upper_bound)]
    # print("Outliers:\n")
print(outlier.columns)

### Check dublicateds and drop duplicateds


In [ ]:
# TODO: Kolla efter duplikate
dups=df.duplicated()
print(dups)

df=df.drop_duplicates()

dropted_dups=df.duplicated()
print(dropted_dups)




### Show data with bar chart

In [ ]:
import matplotlib.pyplot as plt

def bar_subplots(dataFrame, columns):
    n = len(columns)
    fig, axes = plt.subplots(n, 1, figsize=(8, 3*n))

    for ax, col in zip(axes, columns):
        ax.bar(dataFrame.index, dataFrame[col])
        ax.set_title(col)

    plt.tight_layout()
    plt.show()

bar_subplots(df, df.columns)


### show data with histogram chart


In [ ]:
import matplotlib.pyplot as plt

def hist_subplots(df, columns):
    n = len(columns)
    fig, axes = plt.subplots(n, 1, figsize=(8, 3*n))
    colors = ['red','blue','green','orange','purple','brown','orange','gray','cyan','magenta','yellow','black']
    for ax, col,color in zip(axes, columns,colors):
        ax.hist(df[col],bins=7, edgecolor='black', color=color, alpha=0.7)
        ax.set_title(col,color=color)

    plt.tight_layout()
    plt.show()

hist_subplots(df, df.columns)


### Good quality wine data graphs

In [ ]:
good_quality_wine = df[(df['quality'] >= 7.5)]
good_quality_wine.reset_index(drop=True, inplace=True)

bar_subplots(good_quality_wine, good_quality_wine.columns)
hist_subplots(good_quality_wine, good_quality_wine.columns)


### Machine learning

In [ ]:
%pip install scikit-learn

print("✅ scikit-learn installerad!")

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.metrics import (
    confusion_matrix, roc_curve, auc,
    precision_recall_curve
)

from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

print(" Bibliotek importerade!")
print("Vi använder sklearn - världens enklaste ML-bibliotek")

In [ ]:

print(" VÄLJ DATA FÖR MODELLEN")
print("=" * 60)

# TARGET (vad vi vill förutsäga)
Y = df['quality']
print(f" Target (Y): quality")
print(f"   → Detta ska modellen lära sig förutsäga\n")

X = df.loc[:, df.columns != 'quality']

print(f"\n Data shape:")
print(f"   X: {X.shape} (rader, kolumner)")
print(f"   Y: {Y.shape} (rader)")

### Classifing

In [ ]:
# Skapa en kategori:
# 1 = hög temperatur
# 0 = låg temperatur
df["Good_quality"] = (df["quality"] > 6.5).astype(float)

print(df[["quality", "Good_quality"]].head(20))

In [ ]:
x_clf =df[[df.loc[:, df.columns != 'quality']]]
y_clf = df[['quality']]

X_train, X_test, Y_train, Y_test = train_test_split(
    x_clf, y_clf, test_size=0.2, random_state=42)

### Logical regression

In [65]:
log_reg = LogisticRegression(
    class_weight='balanced',
    max_iter=1000,
    solver="liblinear"   
)
log_reg.fit(X_train, Y_train)

y_pred_log = log_reg.predict(X_test)

print("\nLogistisk regression")
print("Accuracy:", accuracy_score(Y_test, y_pred_log))

precision = precision_score(Y_test, y_pred_log)
recall = recall_score(Y_test, y_pred_log)

print("Precision:", precision)
print("Recall:", recall)

/usr/local/lib/python3.14/site-packages/sklearn/linear_model/_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(



Logistisk regression
Accuracy: 0.5367647058823529


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [ ]:
cm_log = confusion_matrix(Y_test, y_pred_log)

sns.heatmap(cm_log, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix – Logistic Regression")
plt.xlabel("Prediktion")
plt.ylabel("Sant värde")
plt.show()

### Random forest classifier

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
rf_clf = RandomForestClassifier(
    n_estimators=200,
    random_state=42
)
rf_clf.fit(X_train, Y_train)

y_pred_rf_clf = rf_clf.predict(X_test)

print("\nRandom Forest Classifier")
print("Accuracy:", accuracy_score(Y_test, y_pred_rf_clf))
precision = precision_score(y_true=Y_test, y_pred=y_pred_rf_clf,average='micro')
recall = recall_score(y_true=Y_test, y_pred=y_pred_rf_clf, average='micro')

print("\nRandom Forest Regression")
print("R2:", r2_score(Y_test, y_pred_rf_clf))
print("Precision:", precision)
print("Recall:", recall)

In [ ]:
cm_rf = confusion_matrix(Y_test, y_pred_rf_clf)

sns.heatmap(cm_rf, annot=True, fmt="d", cmap="Greens")
plt.title("Confusion Matrix – Random Forest")
plt.xlabel("Prediktion")
plt.ylabel("Sant värde")
plt.show()

In [ ]:
importances = rf_clf.feature_importances_
features = x_clf.columns

df = pd.DataFrame({'Feature': features, 'Importance': importances})
print(df.sort_values('Importance', ascending=False))

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(Y_test, y_pred_rf_clf, alpha=0.7)

plt.xlabel("Real quality of the wine")
plt.ylabel("Predicted quality of the wine")
plt.title("Real vs predicted – quality of the wine")

# rita en linje som visar 'perfekta' prediktioner
min_val = min(y_pred_rf_clf.min(), y_pred_rf_clf.min())
max_val = max(y_pred_rf_clf.max(), y_pred_rf_clf.max())
plt.plot([min_val, max_val], [min_val, max_val], color='red', linewidth=2)

plt.grid(True)
plt.show()